In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.master("local[1]").appName("AoC_2022_2[2]").getOrCreate()

In [ ]:
strategy_df = spark.read.option("delimiter", " ").schema("opponent string, our string").csv("./data/aoc_2.txt")

In [ ]:
from itertools import chain

opponent_col = F.col("opponent")
our_col = F.col("our")

hand_shape_score = {"A": 1, "B": 2, "C": 3, "X": 1, "Y": 2, "Z": 3}
hand_shape_score_expr = F.create_map([F.lit(x) for x in chain(*hand_shape_score.items())])

strategy_df_remapped = strategy_df.withColumn("opponent", hand_shape_score_expr[opponent_col]).withColumn("our", hand_shape_score_expr[our_col])
strategy_df_remapped

In [ ]:
winning_expr = ((opponent_col % 3) + 1) == our_col

draw_expr = (opponent_col == our_col)

win_score = F.when(winning_expr, F.lit(6)).when(draw_expr, F.lit(3)).otherwise(F.lit(0))
round_score = our_col + win_score

strategy_df_remapped.select(F.sum(round_score).alias("result")).head()

In [ ]:
result_col = F.col("result")

strategy_df_b = strategy_df.withColumnRenamed("our", "result")

shape_to_draw = opponent_col

shape_to_win = (opponent_col % 3) + 1

shape_to_lose = ((opponent_col + 1)) % 3 + 1


our_hand_expr = (F.when(result_col == F.lit("X"), shape_to_lose)
                  .when(result_col == F.lit("Y"), shape_to_draw)
                  .when(result_col == F.lit("Z"), shape_to_win))

(strategy_df_b.withColumn("opponent", hand_shape_score_expr[opponent_col])
             .withColumn("our", our_hand_expr)
             .select(F.sum(round_score).alias("result")).head())